In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import cv2 
import stripes.palette as palette
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('white')

reload(palette)

<module 'stripes.palette' from 'stripes/palette.pyc'>

In [46]:
def create_random_stripe(image_name):
    plt.ioff()
    fig = plt.figure(figsize=(12,2))
    template = np.ones((64,64,3))
    color = np.array((255,255,255))
    img = (template*color).astype('uint8')
    plt.subplot(1,len(palette.agents_list)+4,1)
    plt.imshow(img)
    plt.axis('off')
    plt.subplot(1,len(palette.agents_list)+4,2)
    plt.imshow(img)
    plt.axis('off')

    for i,agent in enumerate(palette.agents_list):
        plt.subplot(1,len(palette.agents_list)+4,3+i)


        if agent != 'BLO':
            index = np.random.randint(0,high=len(palette.points_dict[agent]))
        else:
            index = np.random.choice([0,3,4,5,6])
        template = np.ones((64,64,3))
        color = np.array(palette.points_dict[agent][index])
        img = (template*color).astype('uint8')
        plt.imshow(img)
        plt.title("{}\n{}".format(agent,index))
        plt.axis('off')
        plt.subplots_adjust(wspace=0.2)

    #plt.tight_layout()

    plt.tight_layout()
    plt.savefig(image_name)

    img = cv2.imread(image_name)
    img[0:3,:,:] = 0
    img[:,0:3,:] = 0
    img[:,-3:,:] = 0
    img[-3:,:,:] = 0
    cv2.imwrite(image_name,img)
    plt.close()

In [47]:
!mkdir random_stripes

mkdir: cannot create directory ‘random_stripes’: File exists


In [48]:
path = "random_stripes/"

In [49]:
import os

In [50]:
for i in xrange(22):
    basename = str(i)+'.png'
    full_name = os.path.join(path,basename)
    create_random_stripe(full_name)